## **Step 1: Setting Up the Environment**  
Make sure you have the required libraries installed. Run the following commands in your terminal:

In [ ]:
!pip install torch torchvision matplotlib tqdm

## **Step 2: Loading the Dataset**

We’ll use `torchvision.datasets` to load the **FashinMNIST** dataset (you can replace this with MNIST, FashionMNIST, etc.).

In [ ]:
import torch
import torchvision
import torchvision.transforms as transforms
from torch.utils.data import DataLoader

# Define transformations (resize, normalize, convert to tensors)
transform = transforms.Compose([
    transforms.Resize((224, 224)),  # Resizing images to 224x224
    transforms.ToTensor(),
    transforms.Lambda(lambda x: x.repeat(3, 1, 1)), # Convert to 3 channel
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))  # Normalization
])

# Load FashionMNIST Dataset
train_dataset = torchvision.datasets.FashionMNIST(root='./data', train=True, download=True, transform=transform)
test_dataset = torchvision.datasets.FashionMNIST(root='./data', train=False, download=True, transform=transform)

# Dataloaders
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)

print(f"Number of training samples: {len(train_dataset)}")
print(f"Number of test samples: {len(test_dataset)}")

Number of training samples: 60000
Number of test samples: 10000


## **Step 3: Loading and Modifying the Pre-trained Model**

I will use **ResNet18**, **VGG16** and **ModileNetV2** and modify the final layer to fit the FashionMNIST dataset (10 classes).

In [ ]:
import torchvision.models as models
import torch.nn as nn

### **1. ResNet18:**

In [ ]:
# Load pre-trained ResNet18 model
model_resnet18 = models.resnet18(pretrained=True)

# Modify the final fully connected (fc) layer
num_ftrs_resnet18 = model_resnet18.fc.in_features  # Input features to the fc layer
model_resnet18.fc = nn.Linear(num_ftrs_resnet18, 10)  # 10 classes for FashionMNIST

# Move model to GPU if available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model_resnet18 = model_resnet18.to(device)

### **2. VGG16:**


In [ ]:
import torchvision.models as models
import torch.nn as nn

# Load pre-trained VGG16 model
model_vgg16 = models.vgg16(pretrained=True)

# Modify the final fully connected (fc) layer
num_ftrs_vgg16 = model_vgg16.classifier[6].in_features  # Input features to the fc layer
model_vgg16.classifier[6] = nn.Linear(num_ftrs_vgg16, 10)  # 10 classes for FashionMNIST

# Move model to GPU if available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model_vgg16 = model_vgg16.to(device)

### **3. MobileNetV2:**


In [ ]:
# Load pre-trained MobileNetV2 model
model_mobilenet_v2 = models.mobilenet_v2(pretrained=True)

# Modify the final fully connected (fc) layer
num_ftrs_mobilenet_v2 = model_mobilenet_v2.classifier[1].in_features  # Input features to the fc layer
model_mobilenet_v2.classifier[1] = nn.Linear(num_ftrs_mobilenet_v2, 10)  # 10 classes for FashionMNIST

# Move model to GPU if available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model_mobilenet_v2 = model_mobilenet_v2.to(device)

## **Step 4: Defining the Loss Function and Optimizer**

In [ ]:
import torch.optim as optim

# Loss function
criterion = nn.CrossEntropyLoss()

In [ ]:
# Optimizer: ResNet18
optimizer_resnet18 = optim.Adam(model_resnet18.parameters(), lr=0.001)

In [ ]:
# Optimizer: VGG16
optimizer_vgg16 = optim.Adam(model_vgg16.parameters(), lr=0.001)

In [ ]:
# Optimizer: MobileNetV2
optimizer_mobilenet_v2 = optim.Adam(model_mobilenet_v2.parameters(), lr=0.001)

## **Step 5: Training the Models**

In [ ]:
from tqdm import tqdm

# Training loop
def train_model(model, train_loader, criterion, optimizer, device, epochs=5):
    model.train()
    for epoch in range(epochs):
        running_loss = 0.0
        for images, labels in tqdm(train_loader, desc=f"Epoch {epoch+1}/{epochs}"):
            images, labels = images.to(device), labels.to(device)

            # Forward pass
            outputs = model(images)
            loss = criterion(outputs, labels)

            # Backward pass
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

            running_loss += loss.item()
        print(f"Epoch {epoch+1}, Loss: {running_loss/len(train_loader):.4f}")

In [ ]:
# Run training on ResNet18
train_model(model_resnet18, train_loader, criterion, optimizer_resnet18, device, epochs=5)

Epoch 1/5: 100%|██████████| 1875/1875 [04:00<00:00,  7.78it/s]


Epoch 1, Loss: 0.7488


Epoch 2/5: 100%|██████████| 1875/1875 [03:53<00:00,  8.02it/s]


Epoch 2, Loss: 0.3928


Epoch 3/5: 100%|██████████| 1875/1875 [03:53<00:00,  8.03it/s]


Epoch 3, Loss: 0.3417


Epoch 4/5: 100%|██████████| 1875/1875 [03:53<00:00,  8.03it/s]


Epoch 4, Loss: 0.3029


Epoch 5/5: 100%|██████████| 1875/1875 [03:54<00:00,  8.00it/s]

Epoch 5, Loss: 0.2745


In [ ]:
# Run training on VGG16
train_model(model_vgg16, train_loader, criterion, optimizer_vgg16, device, epochs=5)

In [ ]:
# Run training on MobileNetV2
train_model(model_mobilenet_v2, train_loader, criterion, optimizer_mobilenet_v2, device, epochs=5)

Epoch 1/5: 100%|██████████| 1875/1875 [04:37<00:00,  6.77it/s]


Epoch 1, Loss: 0.6928


Epoch 2/5: 100%|██████████| 1875/1875 [04:35<00:00,  6.82it/s]


Epoch 2, Loss: 0.4812


Epoch 3/5: 100%|██████████| 1875/1875 [04:34<00:00,  6.83it/s]


Epoch 3, Loss: 0.4343


Epoch 4/5: 100%|██████████| 1875/1875 [04:33<00:00,  6.85it/s]


Epoch 4, Loss: 0.4123


Epoch 5/5: 100%|██████████| 1875/1875 [04:34<00:00,  6.84it/s]

Epoch 5, Loss: 0.4134


## **Step 6: Evaluating the Model**

Now, let’s evaluate the model’s accuracy on the test dataset:

In [ ]:
def evaluate_model(model, test_loader, device):
    model.eval()
    correct = 0
    total = 0
    with torch.no_grad():
        for images, labels in test_loader:
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)
            _, predicted = torch.max(outputs, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
    accuracy = 100 * correct / total
    print(f"Test Accuracy: {accuracy:.2f}%")
    return accuracy

In [ ]:
# Evaluate ResNet18
test_accuracy = evaluate_model(model_resnet18, test_loader, device)

In [ ]:
# Evaluate VGG16
test_accuracy = evaluate_model(model_vgg16, test_loader, device)

In [ ]:
# Evaluate MobileNetV2
test_accuracy = evaluate_model(model_mobilenet_v2, test_loader, device)

## **Step 8: Report Accuracy**
- Learning Rate: 0.001
- Batch Size: 32

| **Model**           | **Test Accuracy (%)** |
|----------------------|-----------------------|
| ResNet18            | 93.24%                  |
| VGG16               | 89.85%                  |
| MobileNetV2         | 89.62%                  |


- Learning Rate: 0.05
- Batch Size: 32


| **Model**           | **Test Accuracy (%)** |
|----------------------|-----------------------|
| ResNet18            | 90.80%                  |
| VGG16               | 10.00%                 |
| MobileNetV2         | 84.92%                |

- Learning Rate: 0.05
- Batch Size: 64

| **Model**           | **Test Accuracy (%)** |
|----------------------|-----------------------|
| ResNet18            | 89.51%                 |
| VGG16               | 90.80%                  |
| MobileNetV2         | 83.24%                  |

- Learning Rate: 0.05
- Batch Size: 128

| **Model**           | **Test Accuracy (%)** |
|----------------------|-----------------------|
| ResNet18            | 86.05%                |
| VGG16               | 10.00%                  |
| MobileNetV2         | 77.10%                  |